In [5]:
import warnings
import pandas as pd
import hashlib
import jieba
import re

from tqdm import tqdm
from spiders.create_db import connect

warnings.filterwarnings('ignore')

ERROR! Session/line number was not unique in database. History logging moved to new session 688


md5加密url

In [6]:
data = 'https://news.sina.com.cn/c/2020-10-07/doc-iivhvpwz0689607.shtml'
ans = hashlib.md5(data.encode(encoding='UTF-8')).hexdigest()
int(ans[8:-8], 16)

15356930010969299234

In [7]:
conn = connect()
table = pd.read_sql("select * from sina limit 500", conn)
table

,id,title,content,link,urls
0,3928515898915405,特朗普复出演讲信口开河“向中国要赔偿”,"原标题:特朗普复出演讲信口开河“向中国要赔偿”,美国网友:害死数十万美国人的是你! [环球...",https://news.sina.com.cn/c/2021-06-06/doc-ikqc...,http://english.sina.com/ http://eladies.sina.c...
1,6578924263750113,林郑昨晚劝市民，这话都说了！,"原标题:林郑昨晚劝市民,这话都说了! 香港立法会教育事务委员会7月3日召开会议,讨论幼儿园...",https://news.sina.com.cn/c/2020-07-05/doc-iirc...,http://english.sina.com/ http://eladies.sina.c...
2,13207172530879708,阿富汗“变天” 考验印度的时候到了,"原标题:阿富汗“变天”,考验印度的时候到了 英国广播公司网站8月17日发表题为《塔利班的胜...",https://news.sina.com.cn/o/2021-08-20/doc-ikqc...,http://english.sina.com/ http://eladies.sina.c...
3,13383836803546473,特朗普会见刘鹤：感谢中方整类列管芬太尼类物质,"原标题:美国总统特朗普会见刘鹤 新华社华盛顿4月4日电(记者颜亮 韩洁)当地时间4月4日,...",https://news.sina.com.cn/2019-04-05/doc-ihvhiq...,http://english.sina.com/ http://eladies.sina.c...
4,14552070335599651,中央关注！孙春兰连夜赴吉林，当地6名官员密集被免,"原标题:中央关注!孙春兰连夜赴吉林,当地6名官员密集被免 来源:政知圈 撰文 | 余晖...",https://news.sina.com.cn/c/2020-05-17/doc-iirc...,http://english.sina.com/ http://eladies.sina.c...
...,...,...,...,...,...
495,1194181248054534402,不欢迎前美国驻成都总领事夫人言论引争议 他这样说,"原标题:“不欢迎庄祖宜”言论引争议,前美驻成都总领馆主厨陈涛回应 (观察者网讯)近期,美国...",https://news.sina.com.cn/c/2020-07-28/doc-iivh...,http://english.sina.com/ http://eladies.sina.c...
496,1194317703306178228,辽宁高院对加拿大籍上诉人罗伯特-劳埃德-谢伦伯格走私毒品一案二审宣判,原标题:辽宁省高级人民法院对加拿大籍上诉人罗伯特·劳埃德·谢伦伯格走私毒品一案二审宣判 “...,https://news.sina.com.cn/c/2021-08-10/doc-ikqc...,http://english.sina.com/ http://eladies.sina.c...
497,1195078446291550668,习近平宣布：中国两年内将提供20亿美元国际援助,"原标题:习主席宣布:中国两年内将提供20亿美元国际援助 北京时间18日晚,国家主席习近平在...",https://news.sina.com.cn/c/xl/2020-05-18/doc-i...,http://edu.sina.com.cn/ http://corp.sina.com.c...
498,1196003080132089630,俄国防部：俄联邦武装力量当日所有任务均已完成,"当地时间24日,俄罗斯国防部表示,为俄罗斯联邦武装力量24日设定的所有任务都已完成。(总台记...",https://news.sina.com.cn/w/2022-02-25/doc-imcw...,http://english.sina.com/ http://eladies.sina.c...


In [8]:
i = 0


def judge(x):
    global i
    ans = hashlib.md5(x['link'].encode(encoding='UTF-8')).hexdigest()
    if x['id'] != int(ans[8:-8], 16):
        print(x['link'])
        print(x['id'])
        i += 1


table.apply(lambda x: judge(x), axis=1)
i

0

分词

In [9]:
jieba.load_userdict('../cache/user_dict.txt')
stopwords = [line.strip() for line in open('../cache/stopwords.txt', 'r', encoding='utf-8').readlines()]
tqdm.pandas(desc="分词")

eng_pattern = re.compile("[A-Za-z]+", re.I)
clean_pattern = re.compile(r'[A-Za-z\d_.!+-=—,$%^，。？、~@#￥…&*《》<>「」{}【】()/]', re.I)


def cut_sentence(sentence):
    # 将英文分词
    eng_words = re.findall(eng_pattern, sentence)
    eng_words = [word for word in eng_words if not word.isdigit()]
    english_words = list(set(eng_words))
    # 中文分词
    chinese_sentence = re.sub(clean_pattern, '', sentence).replace(' ', '')
    chinese_words = jieba.cut_for_search(chinese_sentence)
    chinese_words = list(set(chinese_words))
    all_words = english_words + chinese_words
    return [word for word in all_words if word not in stopwords and 1 < len(word) <= 20 and not word.isdigit()]


table['all_words'] = table['content'].progress_apply(lambda x: cut_sentence(x))
table['all_words']

分词: 100%|██████████| 500/500 [00:09<00:00, 53.72it/s]


0      [损失, 反华, 首付, 卡罗, 言中, 在发言中, 二次, 集体, 国会山, 措施, 信口...
1      [Ip, Regina, Facebook, 大声, 对此, 难以置信, 行政长官, 内容,...
2      [鲍里斯, 政治学, 约翰, 紧张, 阿姆, 治病, 胜利, 治学, 形式, 基斯, 边界,...
3      [珀杜, 中方, 商务部长, 高级顾问, 财政部长, 国人, 经贸, 纳等, 经济, 有成,...
4      [wepolitics, ID, 免职, 民警, 多个, 患者, 接机, 岳晓燕, 肺炎, ...
                             ...                        
495    [焦点人物, 中方, 与此, 不顺, 关闭, 暗指, 近日, 健健康康, 发布, 斯敦, 纳...
496    [SN, 维持原判, 埃德, 甲基, 性质, 公众, 审判程序, 原判, 民法, 伙同他人,...
497    [携起手来, 斗争, 仓库, 经济, 视频, 通道, 团结合作, 第届, 推进, 主义, 设...
498    [Ingrida, Polikha, Kaja, CNN, Jens, Kallas, Ig...
499    [Kill, CEO, CDN, Kamel, UberPop, Cutco, Switch...
Name: all_words, Length: 500, dtype: object

获得tf,idf

In [10]:
import pickle

from gensim.summarization.bm25 import BM25

corpus = table['content']

bm25 = BM25(table['all_words'])

with open("../cache/idf_dic.pickle", mode="wb") as f:
    pickle.dump(bm25.idf, f)

tmp = pd.DataFrame()
tmp['id'] = table['id']
tmp['tf'] = bm25.doc_freqs
with open("../cache/tf_df.pickle", mode="wb") as f:
    pickle.dump(tmp, f)
tmp

,id,tf
0,3928515898915405,"{'损失': 1, '反华': 1, '首付': 1, '卡罗': 1, '言中': 1, ..."
1,6578924263750113,"{'Ip': 1, 'Regina': 1, 'Facebook': 1, '大声': 1,..."
2,13207172530879708,"{'鲍里斯': 1, '政治学': 1, '约翰': 1, '紧张': 1, '阿姆': 1..."
3,13383836803546473,"{'珀杜': 1, '中方': 1, '商务部长': 1, '高级顾问': 1, '财政部长..."
4,14552070335599651,"{'wepolitics': 1, 'ID': 1, '免职': 1, '民警': 1, '..."
...,...,...
495,1194181248054534402,"{'焦点人物': 1, '中方': 1, '与此': 1, '不顺': 1, '关闭': 1..."
496,1194317703306178228,"{'SN': 1, '维持原判': 1, '埃德': 1, '甲基': 1, '性质': 1..."
497,1195078446291550668,"{'携起手来': 1, '斗争': 1, '仓库': 1, '经济': 1, '视频': 1..."
498,1196003080132089630,"{'Ingrida': 1, 'Polikha': 1, 'Kaja': 1, 'CNN':..."


获得tfidf

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tqdm.pandas(desc="tfidf")
table['docs'] = table['all_words'].progress_apply(lambda x: " ".join(x))

docs = table['docs'].tolist()
vectorizer = TfidfVectorizer()
vectorizer.fit(docs)
matrix = vectorizer.transform(docs).toarray()
name = vectorizer.get_feature_names_out()

tfidf: 100%|██████████| 500/500 [00:00<00:00, 100054.96it/s]


保存tfidf

In [12]:
import pickle

tfidf_list = []


def get_tfidf():
    for i in tqdm(range(0, matrix.shape[0]), desc="生成tfidf df"):
        t = pd.Series(matrix[i, :], name)
        t = t[t != 0]
        # 选择100个tfidf值最大的词作为这篇文档的关键词
        t = t.nlargest(100)
        tfidf_list.append(t.to_dict())
        del t


get_tfidf()
table['tfidf'] = tfidf_list
with open("../cache/tfidf_df.pickle", mode="wb") as f:
    pickle.dump(table[['id', 'tfidf']], f)

del table['tfidf']

生成tfidf df: 100%|██████████| 500/500 [00:00<00:00, 1111.28it/s]


计算PageRank

In [13]:
import networkx as nx

digraph = nx.Graph()
links = table['link'].tolist()

for link in links:
    digraph.add_node(link)


def add_edges(x):
    for url in x['urls'].split(' '):
        if url in links:
            digraph.add_edge(x['link'], url)


table.apply(lambda x: add_edges(x), axis=1)
pr_dic = nx.pagerank(digraph)
with open('../cache/pr.pickle', mode='wb') as f:
    pickle.dump(pr_dic, f)
len(pr_dic)

500

In [14]:
index = table['id'].tolist()[0:5]

sql = "select * from sina where id in {index}".format(index=tuple(index))
sql

'select * from sina where id in (3928515898915405, 6578924263750113, 13207172530879708, 13383836803546473, 14552070335599651)'

查询

In [15]:
conn = connect()
cursor = conn.cursor()
cursor.execute(sql)
data = cursor.fetchall()
cursor.close()
conn.close()
len(data)

5

提取关键词

In [16]:
table['keywords'] = [''] * len(table)
idx = 0

with open('../cache/tfidf_df.pickle', mode='rb') as f:
    tfidf_df = pickle.load(f)

for value in tfidf_df['tfidf']:
    table._set_value(idx, 'keywords', list(value.keys()))
    idx += 1

table['keywords']

0      [七零八, 七零八落, 信口, 信口开河, 借新冠, 先说, 北卡罗, 北卡罗来纳州, 卡罗...
1      [facebook, ip, regina, 不忍, 为林, 人站, 假仁假义, 假手, 单...
2      [伦杜, 兰开斯, 兰开斯特, 关键作用, 参与者, 政治学, 数以千计, 治学, 治病, ...
3      [九轮, 农业部长, 努钦, 口信, 同习, 商务部长, 圆形, 富有成效, 对习, 希泽,...
4      [三姐, 丰满, 丰满区, 任双, 免政知, 免月, 刘世君, 刘世君据, 刘春成, 刘艺龙...
                             ...                        
495    [一微博, 一时间, 不顺, 主厨, 交情, 何其, 健健康康, 公布出来, 前美, 卑微,...
496    [上诉人, 丙胺, 事实清楚, 加拿大籍, 劳埃德, 原判, 原审, 埃德, 报请, 政协委...
497    [同二, 枢纽, 偿付, 及性, 团结合作, 对口, 携起手来, 清关, 致辞, 起手, 世...
498    [igor, jens, kaja, kallas, polikha, stoltenber...
499    [akamai, cdn, cutco, kamel, kill, redswoosh, s...
Name: keywords, Length: 500, dtype: object

倒排索引

对关键词做倒排索引

In [17]:
all_files = table.explode("keywords")
data_change = all_files.groupby('keywords').groups

index_map = dict()

for item in data_change:
    index_map[item] = table.loc[data_change[item], "id"].tolist()

with open('../cache/index_map.pickle', mode='wb') as f:
    pickle.dump(index_map, f)

index_map

{'abc': [45588900320060257, 177558422321667994, 616455604925836187],
 'aclu': [456404445553442800],
 'acting': [484159448302975965],
 'ad': [760627512670574408],
 'adam': [17321357353514440],
 'adolph': [717823667141827982],
 'adt': [760627512670574408],
 'advisory': [1175265597665519415],
 'adx': [760627512670574408],
 'aei': [86558379671419990],
 'aera': [513446405557873713],
 'agency': [890246207407361007],
 'ais': [726340178345976632],
 'akamai': [1197872923654678295],
 'al': [89898365693953652],
 'alan': [303999745143272056],
 'alexandria': [1100978128246061478],
 'alloys': [89898365693953652],
 'analytics': [717113915694702246],
 'andrew': [456404445553442800],
 'angela': [1116022794477685107],
 'ani': [687082392843072285],
 'ap': [693575031800931400],
 'apl': [868152566308616299],
 'app': [71559255030646824,
  86327398134978235,
  831505839577155721,
  928194736944617403],
 'ara': [1131234217413691492],
 'armour': [118807871537547600],
 'arseniy': [888798029461228690],
 'ashley'